In [1]:
# 导入同花顺包和登录同花顺
import datetime as dt
import pandas as pd
from numpy import *
from iFinDPy import *
THS_iFinDLogin("gdzq6888", "199212")

D:\Anaconda\Lib\site-packages\iFinDPy.pth


0

# 多头alpha

## 个股截面离散度

In [2]:
data = THS_DP('index_weight','2023-02-01;000906.SH','date:Y,thscode:Y,security_name:Y,weight:Y').data

In [3]:
# 提取股票代码
stock_code = list(data['thscode'])

In [50]:
def split(list_a, chunk_size):
    for i in range(0, len(list_a), chunk_size):
        yield list_a[i:i + chunk_size]

In [51]:
# 将stock code等分为10份，因为每次访问有上限
stock_code2 = list(split(stock_code,10))

In [59]:
# 获得股票近10年的每日的每半小时价格
data = pd.DataFrame()
for i in stock_code2:
    a = THS_HF(i,'close','Fill:Original,Interval:30','2013-01-31 09:00:00','2023-02-01 17:00:00').data
    data=pd.concat([data,a])

In [110]:
data

,time,thscode,close,year,month,date,lag,return,daily std
0,2013-01-31 10:30,600519.SH,179.18,2013,01,2013-01-31,178.20,0.005499,0.006588
1,2013-01-31 11:00,600519.SH,180.25,2013,01,2013-01-31,179.18,0.005972,0.006588
2,2013-01-31 11:30,600519.SH,179.22,2013,01,2013-01-31,180.25,-0.005714,0.006588
3,2013-01-31 13:30,600519.SH,179.54,2013,01,2013-01-31,179.22,0.001786,0.006588
4,2013-01-31 14:00,600519.SH,179.93,2013,01,2013-01-31,179.54,0.002172,0.006588
...,...,...,...,...,...,...,...,...,...
12232534,2023-02-01 10:00,002901.SZ,36.01,2023,02,2023-02-01,35.79,0.006147,0.008413
12232535,2023-02-01 10:30,002901.SZ,37.11,2023,02,2023-02-01,36.01,0.030547,0.008413
12232536,2023-02-01 11:00,002901.SZ,36.86,2023,02,2023-02-01,37.11,-0.006737,0.008413
12232537,2023-02-01 11:30,002901.SZ,36.89,2023,02,2023-02-01,36.86,0.000814,0.008413


In [61]:
# 加入月，日
data['year'] = data['time'].apply(lambda x: x.split('-')[0])
data['month'] = data['time'].apply(lambda x: x.split('-')[1])

In [72]:
data['date'] = data['time'].str[:10]

F:\Users\Xuwei\AppData\Local\Temp\ipykernel_48704\650977868.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = data['time'].str[:10]


In [64]:
# 计算回报
data['lag'] = data.groupby(['thscode'])['close'].shift(1)

In [65]:
data['return'] = (data['close']-data['lag'])/data['lag']

In [67]:
data = data.dropna()

In [80]:
# 计算日标准差
std = data.groupby(['date'])['return'].std().reset_index()
std = std.rename(columns={'return':'daily std'})
data = pd.merge(data,std,on='date',how='left')
std['month'] = std['date'].str[:7]

In [97]:
# 得到离散度
std_monthly = std.groupby('month')['daily std'].mean().reset_index()
std_monthly = std_monthly.rename(columns={'daily std':'monthly std'})

In [109]:
# 计算高/低离散度
list(std_monthly['monthly std'])[-1]>mean(list(std_monthly['monthly std'])[:-1])

False